In [2]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler

In [3]:
# Configurações para visualização
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = (12, 8)

# Carregando o dataset
print("Carregando o dataset...")
df = pd.read_csv('datasets/california_housing.csv')

Carregando o dataset...


/tmp/ipykernel_46963/2342824143.py:2: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [4]:
# Verificando informações gerais
print("\n--- Informações do Dataset ---")
print(f"Formato do dataset: {df.shape}")
print("\nPrimeiras 5 linhas:")
print(df.head())

print("\nInformações do dataset:")
df.info()

print("\nEstatísticas descritivas:")
print(df.describe())


--- Informações do Dataset ---
Formato do dataset: (20640, 9)

Primeiras 5 linhas:
   MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2  7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3  5.6431      52.0  5.817352   1.073059       558.0  2.547945     37.85   
4  3.8462      52.0  6.281853   1.081081       565.0  2.181467     37.85   

   Longitude  MedHouseVal  
0    -122.23        4.526  
1    -122.22        3.585  
2    -122.24        3.521  
3    -122.25        3.413  
4    -122.25        3.422  

Informações do dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MedInc       20640 non-null  float64
 1   HouseAge     20640 non-null

In [5]:
# Verificando valores ausentes
print("\n--- Verificação de Valores Ausentes ---")
missing_values = df.isnull().sum()
print(missing_values)

# Se houver valores ausentes, tratar
if missing_values.sum() > 0:
    print("Tratando valores ausentes...")
    # Preenchendo valores numéricos ausentes com a média
    for col in df.select_dtypes(include=[np.number]).columns:
        if df[col].isnull().sum() > 0:
            df[col].fillna(df[col].mean(), inplace=True)


--- Verificação de Valores Ausentes ---
MedInc         0
HouseAge       0
AveRooms       0
AveBedrms      0
Population     0
AveOccup       0
Latitude       0
Longitude      0
MedHouseVal    0
dtype: int64


Análise exploratória


In [7]:
# Distribuição das variáveis
print("Gerando histogramas para visualizar a distribuição de cada variável...")
plt.figure(figsize=(20, 15))
df.hist(bins=30, figsize=(20, 15))
plt.suptitle('Distribuição das Variáveis', size=24)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.savefig('histogramas_distribuicao.png')
plt.close()

# Distribuição da variável alvo
print("Analisando a distribuição da variável alvo...")
plt.figure(figsize=(12, 6))
sns.histplot(df['MedHouseVal'], kde=True)
plt.title('Distribuição do Valor Mediano das Casas', size=16)
plt.xlabel('Valor Mediano ($)')
plt.ylabel('Frequência')
plt.savefig('distribuicao_valor_casas.png')
plt.close()

Gerando histogramas para visualizar a distribuição de cada variável...
Analisando a distribuição da variável alvo...


<Figure size 2000x1500 with 0 Axes>

In [11]:
# Distribuição geográfica
print("Gerando mapa de distribuição geográfica dos preços...")
plt.figure(figsize=(12, 10))
plt.scatter(df['Longitude'], df['Latitude'], alpha=0.4, 
            c=df['MedHouseVal'], cmap='viridis', s=10)
plt.colorbar(label='Valor Mediano ($)')
plt.title('Distribuição Geográfica dos Preços das Casas na Califórnia', size=16)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.savefig('mapa_precos.png')
plt.close()

# Análise de Correlação
print("Gerando matriz de correlação...")
correlation_matrix = df.corr()
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matriz de Correlação', size=16)
plt.savefig('matriz_correlacao.png')
plt.close()

Gerando mapa de distribuição geográfica dos preços...
Gerando matriz de correlação...


In [14]:
# Correlação com a variável alvo
print("\nCorrelações com a variável 'MedHouseVal':")
correlations = correlation_matrix['MedHouseVal'].sort_values(ascending=False)
print(correlations)

# Pair plot para variáveis mais correlacionadas
print("Gerando pair plot das variáveis mais correlacionadas...")
top_corr_features = correlations.index[1:5]  # Excluindo a própria median_house_value e limitando a 4
sns.pairplot(df[list(top_corr_features) + ['MedHouseVal']], 
             height=2.5, plot_kws={'alpha': 0.6})
plt.suptitle('Pair Plot das Variáveis Mais Correlacionadas', y=1.02, size=16)
plt.savefig('pair_plot.png')
plt.close()


Correlações com a variável 'MedHouseVal':
MedHouseVal    1.000000
MedInc         0.688075
AveRooms       0.151948
HouseAge       0.105623
AveOccup      -0.023737
Population    -0.024650
Longitude     -0.045967
AveBedrms     -0.046701
Latitude      -0.144160
Name: MedHouseVal, dtype: float64
Gerando pair plot das variáveis mais correlacionadas...


In [15]:
# Identificação de Outliers
print("Gerando boxplot para identificar outliers...")
plt.figure(figsize=(15, 10))
df.boxplot()
plt.title('Boxplot para Identificação de Outliers', size=16)
plt.xticks(rotation=45)
plt.savefig('boxplot_outliers.png')
plt.close()

Gerando boxplot para identificar outliers...


Pré-processamento de dados

In [16]:
# Separação de features e target
print("Separando features e target...")
X = df.drop('MedHouseVal', axis=1)
y = df['MedHouseVal']

# Padronização dos dados
print("Padronizando os dados...")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

# Divisão em conjuntos de treino e teste
print("Dividindo em conjuntos de treino e teste...")
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
print(f"Dimensões do conjunto de treino: {X_train.shape}")
print(f"Dimensões do conjunto de teste: {X_test.shape}")

Separando features e target...
Padronizando os dados...
Dividindo em conjuntos de treino e teste...
Dimensões do conjunto de treino: (16512, 8)
Dimensões do conjunto de teste: (4128, 8)


Modelagem

In [17]:
# Função para avaliar modelos
def evaluate_model(model_name, model, X_train, X_test, y_train, y_test):
    print(f"\nAvaliando o modelo: {model_name}")
    
    # Treinando o modelo
    model.fit(X_train, y_train)
    
    # Fazendo previsões
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # Calculando métricas
    train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    train_mae = mean_absolute_error(y_train, y_pred_train)
    test_mae = mean_absolute_error(y_test, y_pred_test)
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    
    # Cross-validation score
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')
    cv_mean = np.mean(cv_scores)
    
    # Exibindo resultados
    print(f"Métricas de Treino: RMSE = {train_rmse:.2f}, MAE = {train_mae:.2f}, R² = {train_r2:.4f}")
    print(f"Métricas de Teste: RMSE = {test_rmse:.2f}, MAE = {test_mae:.2f}, R² = {test_r2:.4f}")
    print(f"Média R² na validação cruzada (5-fold): {cv_mean:.4f}")
    
    # Plotando valores reais vs preditos
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred_test, alpha=0.6)
    plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--')
    plt.xlabel('Valores Reais')
    plt.ylabel('Valores Preditos')
    plt.title(f'Valores Reais vs. Preditos - {model_name}')
    plt.savefig(f'predicoes_{model_name.lower().replace(" ", "_")}.png')
    plt.close()
    # Plotando resíduos
    residuals = y_test - y_pred_test
    plt.figure(figsize=(10, 6))
    plt.scatter(y_pred_test, residuals, alpha=0.6)
    plt.axhline(y=0, color='r', linestyle='--')
    plt.xlabel('Valores Preditos')
    plt.ylabel('Resíduos')
    plt.title(f'Análise de Resíduos - {model_name}')
    plt.savefig(f'residuos_{model_name.lower().replace(" ", "_")}.png')
    plt.close()
    
    return {
        'model': model_name,
        'train_rmse': train_rmse,
        'test_rmse': test_rmse,
        'train_mae': train_mae,
        'test_mae': test_mae,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'cv_r2': cv_mean
    }

# Lista para armazenar resultados
results = []

In [18]:
# 1. Regressão Linear
print("\nTreinando Regressão Linear...")
lr_model = LinearRegression()
lr_results = evaluate_model("Regressão Linear", lr_model, X_train, X_test, y_train, y_test)
results.append(lr_results)

# Analisando os coeficientes
coefs = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': lr_model.coef_
}).sort_values('Coefficient', ascending=False)

print("\nCoeficientes da Regressão Linear:")
print(coefs)

plt.figure(figsize=(12, 8))
sns.barplot(x='Coefficient', y='Feature', data=coefs)
plt.title('Coeficientes da Regressão Linear', size=16)
plt.savefig('coeficientes_regressao_linear.png')
plt.close()



Treinando Regressão Linear...

Avaliando o modelo: Regressão Linear
Métricas de Treino: RMSE = 0.72, MAE = 0.53, R² = 0.6126
Métricas de Teste: RMSE = 0.75, MAE = 0.53, R² = 0.5758
Média R² na validação cruzada (5-fold): 0.6115

Coeficientes da Regressão Linear:
      Feature  Coefficient
0      MedInc     0.852382
3   AveBedrms     0.371132
1    HouseAge     0.122382
4  Population    -0.002298
5    AveOccup    -0.036624
2    AveRooms    -0.305116
7   Longitude    -0.868927
6    Latitude    -0.896635
